# Roguelike Universe - Link Generation

We can generate the inspiration network based on our data.

In [1]:
import os
import io
import json
import pandas as pd

def read_json(path):
    data = ''
    with io.open(path, 'r', encoding='utf-8') as f:
        data = json.loads(f.read())
#         print(__message('Loaded {}'.format(path)))
    return data
    
def save_json(path, data):
    with io.open(path, 'w', encoding='utf-8') as f:
        try:
            output = json.dumps(data, indent=2, ensure_ascii=False)
            f.write(output)
        except UnicodeEncodeError:
            f.write(output.encode('utf-8'))
#     print(__message('Written to {}'.format(path)))
    
def __success(text):
    return '  (SUCC) {}'.format(text).encode('utf-8')
    
def __failure(text):
    return '!!FAIL!! {}'.format(text).encode('utf-8')
    
def __warning(text):
    return '??WARN?? {}'.format(text).encode('utf-8')
    
def __message(text):
    return '   |MSG| {}'.format(text).encode('utf-8')

## RogueTemple Influences

The data already comes with existing influences documented by the community.

We also load the collected corpus.

In [2]:
roguelikes = pd.read_csv(os.path.join(os.getcwd(), 'roguelikes.csv'), skip_blank_lines=True)
roguelike_names = set(x for x in roguelikes['Name'] if isinstance(x, str))

roguelikelikes = pd.read_csv(os.path.join(os.getcwd(), 'roguelike-likes.csv'), skip_blank_lines=True)
roguelikelike_names = set(x for x in roguelikelikes['Name'] if isinstance(x, str))

video_games = pd.read_json(os.path.join(os.getcwd(), 'games.json'))
video_game_names = set(x for x in video_games['title'] if isinstance(x, str) and not all(d.isdigit() for d in x))

roguelike_corpus = read_json('corpus.json')
roguelikelike_corpus = read_json('corpus-roguelike-like.json')

### RogueTemple Influences

In [3]:
import datetime

roguelike_universe_data = roguelikes.copy(deep=True)
roguelike_universe_data.set_index('Name')

for i, roguelike in roguelike_universe_data.iterrows():
    title = roguelike['Name']
    influences = roguelike['Influences']
    
    if not isinstance(title, str):
        continue
        
    if isinstance(influences, str):
        roguelike['Influences'] = [x.strip() for x in influences.split(',')]
    else:
        roguelike['Influences'] = []
        
    if isinstance(roguelike['Released'], str) and roguelike['Released'].endswith('/00'):
        if roguelike['Released'].startswith('1000'):
            roguelike['Released'] = None
        else:
            roguelike['Released'] = datetime.date(int(roguelike['Released'][:4]), 1, 1)
    else:
        year, month, day = (int(x) for x in roguelike['Released'].split('/'))
        roguelike['Released'] = datetime.date(year, month, day)
        
    if isinstance(roguelike['Updated'], str) and roguelike['Updated'].endswith('/00'):
        if roguelike['Updated'].startswith('1000'):
            roguelike['Updated'] = None
        else:
            roguelike['Updated'] = datetime.date(int(roguelike['Updated'][:4]), 1, 1)
    else:
        year, month, day = (int(x) for x in roguelike['Updated'].split('/'))
        roguelike['Updated'] = datetime.date(year, month, day)
        
# sample of influences
roguelike_universe_data.head(10)

,Name,RogueTemple,Link,Released,Updated,Developer,Theme,Influences
0,100 Rogues,http://roguebasin.roguelikedevelopment.org/ind...,http://www.100rogues.com/,2010-05-06,2010-05-06,Dinofarm Games,Fantasy,[Rogue]
1,1Quest,http://roguebasin.roguelikedevelopment.org/ind...,http://www.ratzngodz.fr,2014-02-20,2015-02-07,Ratz 'N' Godz,Fantasy,"[DCSS, Dominion 4]"
2,3059,http://roguebasin.roguelikedevelopment.org/ind...,https://sites.google.com/site/free3069/3059---...,2005-01-01,2005-06-11,Phr00t,"Science Fiction, Alien Planets, Futuristic",[NetHack]
3,3069,http://roguebasin.roguelikedevelopment.org/ind...,http://sites.google.com/site/free3069/,2009-07-06,2009-10-06,Phr00t,"Science Fiction, Alien Planets, Futuristic",[3059]
4,3079,http://roguebasin.roguelikedevelopment.org/ind...,http://sites.google.com/site/3079game/,2011-10-25,2015-02-13,Phr00t,"Science Fiction, Alien Planets, Futuristic","[3059, 3069, Fallout, Minecraft]"
5,3089,http://roguebasin.roguelikedevelopment.org/ind...,http://3089game.wordpress.com/,2013-02-02,2014-02-27,Phr00t,"Science Fiction, Alien Planets, Futuristic","[3059, 3069, 3079, Borderlands]"
6,7KBRLL,http://roguebasin.roguelikedevelopment.org/ind...,https://sites.google.com/site/7kbrll,2014-07-11,2014-01-01,OMVTW,"humor, psychological","[NetHack, Rogue]"
7,ADOM,http://roguebasin.roguelikedevelopment.org/ind...,http://www.adom.de/,1994-01-01,2018-05-02,Thomas Biskup,Fantasy,[Hack]
8,Advanced Rogue,http://roguebasin.roguelikedevelopment.org/ind...,http://rogue.rogueforge.net/,2005-01-01,2007-08-01,"Michael Morgan, Ken Dalka, Roguelike Restorati...",NaN,"[Rogue, Super-Rogue]"
9,AGB Hack,http://roguebasin.roguelikedevelopment.org/ind...,http://www.freewebs.com/drussell/#GBA,2006-11-26,2006-11-26,Donnie Russell,NaN,[]


### Keyword spotting: matching mentions of other games

In [4]:
import re
from collections import Counter

roguelike_influence = []
roguelikelike_influence = []
other_influence = []

not_games = set([
    'Steam',
])

for i, roguelike in enumerate(roguelike_corpus):
    roguelike_things = Counter()
    roguelikelike_things = Counter()
    other_things = Counter()
    things = []
    
    if isinstance(roguelike['title'], str):
        for webpage in roguelike['text']:
            for paragraph in webpage.split('\n\n'):
                if not paragraph.strip():
                    continue

                current = ''
                for token in re.split('\W', paragraph):
                    if      len(token) > 1 and \
                            re.compile("^[A-Z0-9][\w:']*[\w:']|[A-Z\.]+$").match(token) or \
                            (current and token in ('the', 'of', 'no', 'to')):
                        current += '{} '.format(token)
                    elif current:
                        things.append(current.strip())
                        current = ''

        for x in things:
            if x in roguelike_names and x != roguelike['title']:
                roguelike_things[x] += 1
            elif x in roguelikelike_names and x != roguelike['title']:
                roguelikelike_things[x] += 1
            elif x in video_game_names and x != roguelike['title'] and x not in not_games:
                other_things[x] += 1  
            
    roguelike_influence.append({
        "title": roguelike['title'],
        "influences": roguelike_things
    })
    
    roguelikelike_influence.append({
        "title": roguelike['title'],
        "influences": roguelikelike_things
    })
    
    other_influence.append({
        "title": roguelike['title'],
        "influences": other_things
    })

In [5]:
roguelike_universe_data["Inferred_Roguelike_Influences"] = [x["influences"] for x in roguelike_influence]
roguelike_universe_data["Inferred_Roguelikelike_Influences"] = [x["influences"] for x in roguelikelike_influence]
roguelike_universe_data["Inferred_Other_Influences"] = [x["influences"] for x in other_influence]
roguelike_universe_data.head(10)

,Name,RogueTemple,Link,Released,Updated,Developer,Theme,Influences,Inferred_Roguelike_Influences,Inferred_Roguelikelike_Influences,Inferred_Other_Influences
0,100 Rogues,http://roguebasin.roguelikedevelopment.org/ind...,http://www.100rogues.com/,2010-05-06,2010-05-06,Dinofarm Games,Fantasy,[Rogue],"{'Diablo': 2, 'The Dungeon': 1, 'Rogue': 2}",{},"{'Combat': 1, 'Monsters': 3, 'Ghosts': 1, 'And..."
1,1Quest,http://roguebasin.roguelikedevelopment.org/ind...,http://www.ratzngodz.fr,2014-02-20,2015-02-07,Ratz 'N' Godz,Fantasy,"[DCSS, Dominion 4]",{},{},"{'Android': 3, 'Epic': 1, 'Add': 2}"
2,3059,http://roguebasin.roguelikedevelopment.org/ind...,https://sites.google.com/site/free3069/3059---...,2005-01-01,2005-06-11,Phr00t,"Science Fiction, Alien Planets, Futuristic",[NetHack],"{'3069': 1, '3079': 1, '3089': 1}",{},{}
3,3069,http://roguebasin.roguelikedevelopment.org/ind...,http://sites.google.com/site/free3069/,2009-07-06,2009-10-06,Phr00t,"Science Fiction, Alien Planets, Futuristic",[3059],"{'3059': 2, '3079': 1, '3089': 1}",{},"{'Core': 1, 'Alien': 1, 'There': 1}"
4,3079,http://roguebasin.roguelikedevelopment.org/ind...,http://sites.google.com/site/3079game/,2011-10-25,2015-02-13,Phr00t,"Science Fiction, Alien Planets, Futuristic","[3059, 3069, Fallout, Minecraft]","{'3059': 3, '3069': 3, '3089': 2}",{},"{'Fallout': 1, 'Minecraft': 1, 'One': 1, 'Blit..."
5,3089,http://roguebasin.roguelikedevelopment.org/ind...,http://3089game.wordpress.com/,2013-02-02,2014-02-27,Phr00t,"Science Fiction, Alien Planets, Futuristic","[3059, 3069, 3079, Borderlands]","{'3059': 1, '3069': 1, '3079': 5}",{},"{'Overlord': 8, 'Gentrieve': 1, 'There': 2}"
6,7KBRLL,http://roguebasin.roguelikedevelopment.org/ind...,https://sites.google.com/site/7kbrll,2014-07-11,2014-01-01,OMVTW,"humor, psychological","[NetHack, Rogue]",{'NetHack': 1},{},{}
7,ADOM,http://roguebasin.roguelikedevelopment.org/ind...,http://www.adom.de/,1994-01-01,2018-05-02,Thomas Biskup,Fantasy,[Hack],{},{},"{'Ancient Domains of Mystery': 2, 'Dungeons': ..."
8,Advanced Rogue,http://roguebasin.roguelikedevelopment.org/ind...,http://rogue.rogueforge.net/,2005-01-01,2007-08-01,"Michael Morgan, Ken Dalka, Roguelike Restorati...",NaN,"[Rogue, Super-Rogue]","{'Rogue': 108, 'XRogue': 17, 'Dweller': 2, 'Li...",{},"{'Contact': 1, 'Turbo': 1, 'MAG': 4, 'Ancient ..."
9,AGB Hack,http://roguebasin.roguelikedevelopment.org/ind...,http://www.freewebs.com/drussell/#GBA,2006-11-26,2006-11-26,Donnie Russell,NaN,[],"{'Hack': 1, 'NetHack': 1}",{},{}


In [22]:
import math

roguelikelike_universe_data = roguelikelikes.copy(deep=True)
roguelikelike_universe_data.set_index('Name')
roguelikelike_universe_data['Released'] = roguelikelike_universe_data['Released'].astype(datetime.date)
roguelikelike_universe_data['Updated'] = roguelikelike_universe_data['Updated'].astype(datetime.date)
roguelikelike_universe_data['Influences'] = roguelikelike_universe_data['Influences'].astype(list)

for i, roguelike in roguelikelike_universe_data.iterrows():
    title = roguelike['Name']
    influences = roguelike['Influences']
    
    if not isinstance(title, str):
        continue
        
    if isinstance(influences, str):
        roguelike['Influences'] = [x.strip() for x in influences.split(',')]
    else:
        roguelike['Influences'] = []
        
    if not math.isnan(roguelike['Released']):
        roguelike['Released'] = datetime.date(int(roguelike['Released']), 1, 1)
        
    if not math.isnan(roguelike['Updated']):
        roguelike['Updated'] = datetime.date(int(roguelike['Updated']), 1, 1)
        
# sample of influences
roguelikelike_universe_data.head(10)

,Name,Released,Updated,Developer,Theme,Influences
0,ToeJam & Earl,1991-01-01,NaN,Johnson Voorsanger Productions,Fantasy,[]
1,Diablo,1996-01-01,NaN,Blizzard North,Fantasy,[]
2,Lost Labyrinth,2001-01-01,2011-01-01,Lost Labyrinth,Fantasy,[]
3,Strange Adventures In Infinite Space,2002-01-01,2004-01-01,"Rich Carlson, Iikka Keränen",Space science fiction,[]
4,Weird Worlds: Return to Infinite Space,2005-01-01,2014-01-01,"Rich Carlson, Iikka Keränen",Space science fiction,[]
5,Spelunky,2009-01-01,2012-01-01,"Derek Yu, Andy Hull",Fantasy,[]
6,The Binding of Isaac,2011-01-01,NaN,"Edmund McMillen, Florian Himsl","Surrealistic, horror",[]
7,FTL: Faster Than Light,2012-01-01,NaN,Subset Games,Space science fiction,[]
8,Don't Starve,2013-01-01,NaN,Klei Entertainment,Fantasy,[]
9,Rogue Legacy,2013-01-01,NaN,Cellar Door Games,Fantasy,[]


In [23]:
import re
from collections import Counter

roguelike_influence = []
roguelikelike_influence = []
other_influence = []

not_games = set([
    'Steam',
])

for i, roguelike in enumerate(roguelikelike_corpus):
    roguelike_things = Counter()
    roguelikelike_things = Counter()
    other_things = Counter()
    things = []
    
    if isinstance(roguelike['title'], str):
        for webpage in roguelike['text']:
            for paragraph in webpage.split('\n\n'):
                if not paragraph.strip():
                    continue

                current = ''
                for token in re.split('\W', paragraph):
                    if      len(token) > 1 and \
                            re.compile("^[A-Z0-9][\w:']*[\w:']|[A-Z\.]+$").match(token) or \
                            (current and token in ('the', 'of', 'no', 'to')):
                        current += '{} '.format(token)
                    elif current:
                        things.append(current.strip())
                        current = ''

        for x in things:
            if x in roguelike_names and x != roguelike['title']:
                roguelike_things[x] += 1
            elif x in roguelikelike_names and x != roguelike['title']:
                roguelikelike_things[x] += 1
            elif x in video_game_names and x != roguelike['title'] and x not in not_games:
                other_things[x] += 1  
            
    roguelike_influence.append({
        "title": roguelike['title'],
        "influences": roguelike_things
    })
    
    roguelikelike_influence.append({
        "title": roguelike['title'],
        "influences": roguelikelike_things
    })
    
    other_influence.append({
        "title": roguelike['title'],
        "influences": other_things
    })

In [ ]:
roguelike_universe_data["Inferred_Roguelike_Influences"] = [x["influences"] for x in roguelike_influence]
roguelike_universe_data["Inferred_Roguelikelike_Influences"] = [x["influences"] for x in roguelikelike_influence]
roguelike_universe_data["Inferred_Other_Influences"] = [x["influences"] for x in other_influence]
roguelike_universe_data.head(10)